# caching with functools

In [ ]:
from functools import *
import numpy as np
import urllib

## @cache

In [ ]:
def factorial(n):
    return n * factorial(n-1) if n else 1

In [ ]:
def run():
    factorial(200)
    for _ in range(100):
        factorial(100)

In [ ]:
%%timeit -r 5 -n 10
run()

2.06 ms ± 539 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [ ]:
@cache
def run_c():
    factorial(200)
    for _ in range(100):
        factorial(100)

In [ ]:
%%timeit -r 5 -n 10
run_c()

The slowest run took 3142.82 times longer than the fastest. This could mean that an intermediate result is being cached.
62.8 µs ± 125 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)


## @cached_property

In [ ]:
class Data:
    def __init__(self, *sequence):
        self._data = tuple(sequence)

    @property
    def std(self):
        return np.std(self._data)

In [ ]:
data = Data(1,2,43,5,7,8,9,0)

In [ ]:
%%timeit -n 10000
data.std

16.8 µs ± 3.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
class Data:
    def __init__(self, *sequence):
        self._data = tuple(sequence)

    @cached_property
    def std(self):
        return np.std(self._data)

In [ ]:
data = Data(1,2,43,5,7,8,9,0)

In [ ]:
%%timeit -n 10000
data.std

65.9 ns ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## @lru_cache(maxsize, typed)

In [ ]:
@lru_cache(maxsize=32)
def get_pep(num):
    'Retrieve text of a Python Enhancement Proposal'
    resource = f'https://peps.python.org/pep-{num:04d}'
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [ ]:
get_pep.cache_info()

CacheInfo(hits=294003, misses=8, maxsize=32, currsize=8)

In [ ]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep=get_pep(n)
    print(n,len(pep))

8 119008
290 53070
308 41334
320 21642
8 119008
218 20625
320 21642
279 20092
289 30387
320 21642
9991 9


In [ ]:
%%timeit -n 1000
for _ in range(5):
     get_pep(8)

408 ns ± 125 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
def get_pep_1(num):
    'Retrieve text of a Python Enhancement Proposal'
    resource = f'https://peps.python.org/pep-{num:04d}'
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [ ]:
%%timeit -n 5 -r 3
for _ in range(5):
     get_pep_1(8)

925 ms ± 28.1 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
